## Publish a Triangle Mesh object

This example shows how to convert a mesh file in `obj` format into an Evo geoscience object using the Evo Python SDK.

### Requirements

You must have a Seequent account with the Evo entitlement to use this notebook.

The following parameters must be provided:

- The client ID of your Evo application.
- The callback/redirect URL of your Evo application.

To obtain these app credentials, refer to the [Apps and tokens guide](https://developer.seequent.com/docs/guides/getting-started/apps-and-tokens) in the Seequent Developer Portal.

In [ ]:
from evo.notebooks import ServiceManagerWidget

cache_location = "data"
input_path = f"{cache_location}/input"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    discovery_url="https://discover.api.seequent.com",
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

### Use the Evo Python SDK to create an object client and a data client

In [ ]:
from evo.objects import ObjectAPIClient

# The object client will manage your auth token and Geoscience Object API requests.
object_client = ObjectAPIClient(manager.get_environment(), manager.get_connector())

# The data client will manage saving your data as Parquet and publishing your data to Evo storage.
data_client = object_client.get_data_client(manager.cache)

### Define a helper function

This function assists with viewing the new object in the Evo portal.

In [ ]:
from IPython.display import HTML, display


def build_portal_url(object_metadata):
    """
    Build the URL to view the object in the Evo Portal.

    Args:
        object_metadata (str): The object metadata object returned after the object is created.

    Returns:
        str: The URL to view the object in the Evo Portal.
    """

    hub_url = object_metadata.environment.hub_url
    hub_name = hub_url.split("://")[1].split(".")[0]
    org_id = object_metadata.environment.org_id
    workspace_id = object_metadata.environment.workspace_id
    object_id = object_metadata.id

    url = f"https://evo.seequent.com/{org_id}/workspaces/{hub_name}/{workspace_id}/viewer?id={object_id}"

    display(HTML(f'<a href="{url}" target="_blank">View object in the Evo Portal</a>'))

### Define object metadata

Geoscience object data must conform to a specific object schema. The `evo-schemas` package provides Pydantic models that make it easy to work with the equivalent JSON schemas. 
For this example we'll use v2.1.0 of the triangle mesh schema, via the relevant Pydantic model.

Enter values for these parameters that are required by the object schema.
- `object_name`: The name of the object.
- `object_path`: The file path where the object will be found.
- `object_epsg_code`: (Optional) The EPSG region code that matches the location of your data. Leave as `None` if not required.
- `object_tags`: (Optional) A dictionary of additional tags to be assigned to the object. Leave as `None` is not required.

In [ ]:
from evo_schemas.components import Crs_V1_0_1_EpsgCode

object_name = "Mesh_SDK_demo"
object_path = "Jupyter_Example/Mesh"
object_epsg_code = 32650
object_tags = {"Source": "Jupyter Notebook"}

# Define a coordinate reference system (CRS) for the object.
coordinate_reference_system = Crs_V1_0_1_EpsgCode(epsg_code=object_epsg_code)

### Set the full object path for the final Evo API call.
# NOTE: The object name requires the `.json` extension
full_obj_path = f"{object_path}/{object_name}.json"

### Configure input object data

In [ ]:
import numpy as np

vertices = []
faces = []

relative_path = "data/input/cow.obj"

with open(relative_path, "r") as file:
    for line in file:
        if line.startswith("v "):
            _, x, y, z = line.split()
            vertices.append([float(x), float(y), float(z)])
        elif line.startswith("f "):
            parts = line.split()[1:]
            face = [int(part.split("/")[0]) for part in parts]
            faces.append(face)

vertices_array = np.array(vertices)
faces_array = np.array(faces)
faces_array -= 1

### Create sample locations and triangle data

In [ ]:
import pandas as pd
import pyarrow as pa

schema_v = pa.schema(
    [
        pa.field("X", pa.float64()),
        pa.field("Y", pa.float64()),
        pa.field("Z", pa.float64()),
    ]
)

schema_f = pa.schema([pa.field("X", pa.uint64()), pa.field("Y", pa.uint64()), pa.field("Z", pa.uint64())])

table_v = pa.Table.from_arrays(
    [pa.array(vertices_array[:, i], type=pa.float64()) for i in range(len(schema_v))],
    schema=schema_v,
)
table_f = pa.Table.from_arrays(
    [pa.array(faces_array[:, i], type=pa.uint64()) for i in range(len(schema_f))],
    schema=schema_f,
)
df_v = pd.DataFrame(table_v.to_pandas())
df_f = pd.DataFrame(table_f.to_pandas())

### Coordinates

Create a **coordinates** dataframe and copy the required columns.

In [ ]:
import pandas as pd
from evo_schemas.components import BoundingBox_V1_0_1, Triangles_V1_2_0_Indices, Triangles_V1_2_0_Vertices

vertices_coordinates_df = pd.DataFrame()
vertices_coordinates_df[["x", "y", "z"]] = df_v[["X", "Y", "Z"]]

# Define the bounding box from vertices
vertices_bounding_box = BoundingBox_V1_0_1(
    min_x=vertices_coordinates_df["x"].min(),
    max_x=vertices_coordinates_df["x"].max(),
    min_y=vertices_coordinates_df["y"].min(),
    max_y=vertices_coordinates_df["y"].max(),
    min_z=vertices_coordinates_df["z"].min(),
    max_z=vertices_coordinates_df["z"].max(),
)

mesh_vertices = Triangles_V1_2_0_Vertices.from_dict(data_client.save_dataframe(df_v))
mesh_indices = Triangles_V1_2_0_Indices.from_dict(data_client.save_dataframe(df_f))

### Create a new triangle mesh and publish it to Evo

In [ ]:
from evo_schemas.components import Triangles_V1_2_0
from evo_schemas.objects import TriangleMesh_V2_1_0

from evo.notebooks import FeedbackWidget

# Lastly, assemble the complete geoscience object by combining all previously defined components.
# - The name and UUID are used to identify the object.
# - The UUID is set to None because this is a new object. A new UUID will be assigned by the Evo service.
# - The bounding box defines the spatial extent of the object.
# - The tags provide metadata about the object.
# - The coordinate reference system defines the spatial reference for the object.
# - The locations component contains the coordinates and attributes.

triangle_mesh = TriangleMesh_V2_1_0(
    name=object_name,
    uuid=None,
    bounding_box=vertices_bounding_box,
    tags=object_tags,
    coordinate_reference_system=coordinate_reference_system,
    triangles=Triangles_V1_2_0(vertices=mesh_vertices, indices=mesh_indices),
)

# Upload the Parquet data to Evo.
await data_client.upload_referenced_data(triangle_mesh.as_dict(), FeedbackWidget("Uploading data"))

# Create the geoscience object.
new_pointset_metadata = await object_client.create_geoscience_object(full_obj_path, triangle_mesh.as_dict())

### View the object in the Evo portal

In [ ]:
build_portal_url(new_pointset_metadata)

Success! You now have a new geoscience object in Evo containing your mesh data.

## Summary

In this example, we've completed the following:
* Analysed the vertices and indices and constructed the elements and components required for these properties.
* Converted the input data into Parquet format and saved it to the local cache.
* Combined all of the elements, components and data references into the triangle-mesh schema format.
* Uploaded the Parquet files and the newly assembled object in JSON format to Evo.